In [52]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [53]:
import sqlite3
import numpy as np
from SlideRunner.dataAccess.database import Database
from tqdm import tqdm
from pathlib import Path
import openslide
from sklearn.cluster import KMeans

In [54]:
path = Path('/data/Datasets/EIPH_WSI/')

database = Database()
database.open(str(path/'EIPH.sqlite'))

True

In [62]:
image_size = 1024 / 2

In [66]:
bbox_x = {}
bbox_y = {}

getslides = """SELECT uid, filename FROM Slides"""
for currslide, filename in tqdm(database.execute(getslides).fetchall()):
    database.loadIntoMemory(currslide)

    check = True if 'erliner' in filename else False
    slidetype = 'Berliner Blau/' if check else 'Turnbull Blue/'

    slide_path = path / slidetype / filename

    slide = openslide.open_slide(str(slide_path))
    level = 1#slide.level_count - 1
    level_dimension = slide.level_dimensions[level]
    down_factor = slide.level_downsamples[level]

    classes = {3: 1, 4: 2, 5: 3, 6: 4, 7: 5}
    for id, annotation in database.annotations.items():
        if annotation.labels[0].classId in classes:
            d = 2 * annotation.r / down_factor
            x_min = (annotation.x1 - annotation.r) / down_factor
            y_min = (annotation.y1 - annotation.r) / down_factor
            x_max = x_min + d
            y_max = y_min + d
            
            label = classes[annotation.labels[0].classId]
            if label not in bbox_x:
                bbox_x[label] = []
                bbox_y[label] = []
                
            bbox_x[label].append((x_max-x_min) / image_size)
            bbox_y[label].append((y_max-y_min) / image_size)

  0%|          | 0/24 [00:00<?, ?it/s]

Loading DB into memory ...


  4%|▍         | 1/24 [00:01<00:30,  1.31s/it]

Loading DB into memory ...


  8%|▊         | 2/24 [00:02<00:29,  1.35s/it]

...

Loading DB into memory ...
Loading DB into memory ...


 88%|████████▊ | 21/24 [00:07<00:01,  2.82it/s]

Loading DB into memory ...
Loading DB into memory ...


100%|██████████| 24/24 [00:08<00:00,  2.78it/s]

Loading DB into memory ...


In [67]:
bbox_x.keys()

dict_keys([2, 3, 1, 4, 5])

In [68]:
for key in bbox_x:
    X = np.array(bbox_x[key]).reshape(-1, 1)
    
    kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
    print("Class: {} Centers: {}".format(key, np.sort(kmeans.cluster_centers_.flatten())))

Class: 2 Centers: [0.0442172  0.05931851 0.08028357]
Class: 3 Centers: [0.04464239 0.05973084 0.08166428]
Class: 1 Centers: [0.04173503 0.05518707 0.07439747]
Class: 4 Centers: [0.04571    0.06090824 0.08304216]
Class: 5 Centers: [0.04394687 0.05870508 0.07920171]
